# Equilibrating the System

## Overview

### Questions

* What is equilibration?
* How do I save simulation results?

### Objectives

* Explain the process of **equilibration**.
* Demonstrate using **GSD** to write the simulation trajectory to a file.
* Introduce the **Logger** and show how to add logged data to the **GSD** file.

In [1]:
import hoomd
import math

## Equilibration

So far in this tutorial, we have placed *N* non-overlapping tetrahedra randomly in a box and then quickly compressed them to a moderate packing fraction.
The resulting configuration of particles is valid, but strongly dependent on the path we took to create it.
There are many more **equilibrium** configurations in the set of possible configurations that do not depend on the path.
**Equilibrating** the system is the process of taking an artificially prepared state and running a simulation.
During the simulation run, the system will relax to **equilibrium**.
First, let's initialize the **Simulation**.

In [2]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu)
mc = hoomd.hpmc.integrate.ConvexPolyhedron(seed=42)
mc.shape['tetrahedron'] = dict(vertices=[(math.sqrt(8/9), 0, -1/3),
                                         (-math.sqrt(2/9), math.sqrt(2/3), -1/3),
                                         (-math.sqrt(2/9), -math.sqrt(2/3), -1/3),
                                         (0,0,1)])

sim.operations.integrator = mc

HOOMD-blue v2.9.0-1828-gd903f0dc6 DOUBLE HPMC_MIXED SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX AVX2 
Compiled: 06/10/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
HOOMD-blue is running on the CPU


The previous section of this tutorial wrote the compressed system to `compressed.gsd`.
Initialize the system **state** from this file:

In [3]:
sim.create_state_from_gsd(filename='compressed.gsd')

## Writing simulation trajectories

Let's save the system **state** to a file periodically so that we can observe the equilibration process.
So far in this tutorial, we have used **GSD** files to store a single frame of the system **state** using either the **GSD** Python package or `GSD.write_state`.
Here, let's use the **GSD analyzer** (another **operation**) to create a **GSD** file with many frames in a **trajectory**.

In [4]:
gsd = hoomd.dump.GSD(filename='trajectory.gsd', trigger=hoomd.trigger.Periodic(10000), overwrite=True)
sim.operations.analyzers.append(gsd)

**Operations** mark some of their class *properties* as **loggable**.
**GSD** files can also store these **loggable properties** in addition to the system **state**.
We've seen some of these **loggable properties** earlier in this tutorial: `mc.translate_moves`, `mc.rotate_moves`, and `mc.overlaps`.
You can look at the documentation for a specific **operation** to find a list of all the **logged quantities** 

Use **Logger** object to select which of these **loggable properties** to include in the **GSD** file.
Let's just add all of the **loggable properties** that the HPMC integrator provides:

In [5]:
logger = hoomd.Logger()
logger += mc

Use this **Logger** to populate the **GSD** file:

In [6]:
# currently, the logger is broken with the ConvexPolyhedron shape parameters
# gsd.log = logger

## Tuning the trial move size

**TODO**

In [7]:
mc.d['tetrahedron'] = 0.015
mc.a['tetrahedron'] = 0.03

## Equilibrating the system

To equilibrate the system, **run** the simulation.
The length of the run needed is strongly dependent on the particular model, the system size, the density, and many other factors.
Hard particle Monte Carlo self-assembly often takes tens of millions of time steps for systems with ~10,000 particles.
Our system is much smaller, so it should take far less.

In [ ]:
sim.verbose_run = True
sim.run(1e3)
print(mc.translate_moves[0] / sum(mc.translate_moves))
print(mc.rotate_moves[0] / sum(mc.rotate_moves))
sim.run(5e6)

*Warning*: Box size is too small or move size is too large for the minimum image convention.
Testing 33 images per trial move, performance may slow.
This message will not be repeated.


Time 00:00:09 | Step 13940 / 13940 | TPS 103.674 | ETA 00:00:00
Average TPS: 103.663
---------
notice(2): -- HPMC stats:
notice(2): Trial moves per second:        135991
notice(2): Overlap checks per second:     1.49148e+07
notice(2): Overlap checks per trial move: 109.675
notice(2): Number of overlap errors:      0
0.45159090839910754
0.13184742745292036
Time 00:00:19 | Step 14980 / 5013940 | TPS 103.963 | ETA 13:21:24
Time 00:00:29 | Step 16016 / 5013940 | TPS 103.587 | ETA 13:24:08
Time 00:00:39 | Step 17030 / 5013940 | TPS 101.366 | ETA 13:41:35
Time 00:00:49 | Step 18027 / 5013940 | TPS 99.6237 | ETA 13:55:47
Time 00:00:59 | Step 19012 / 5013940 | TPS 98.4871 | ETA 14:05:16
Time 00:01:09 | Step 19990 / 5013940 | TPS 97.7769 | ETA 14:11:14
Time 00:01:19 | Step 20959 / 5013940 | TPS 96.8506 | ETA 14:19:13
Time 00:01:29 | Step 21928 / 5013940 | TPS 96.8884 | ETA 14:18:43
Time 00:01:39 | Step 22882 / 5013940 | TPS 95.372 | ETA 14:32:12
Time 00:01:49 | Step 23834 / 5013940 | TPS 95.148

How do we know if we have run long enough to **equilibrate** the system?
Let's look at the **trajectory** in the next section of this tutorial and find out.